#### Sobre o Conjunto de Dados
Este é um conjunto de dados público de pedidos do e-commerce brasileiro feitos na loja Olist. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em diversos marketplaces no Brasil.
Esses são dados comerciais reais, que foram anonimizados, e as referências às empresas e parceiros nos textos de avaliação foram substituídas pelos nomes das grandes casas de Game of Thrones.


#### Contexto
Este conjunto de dados foi generosamente fornecido pela Olist, a maior loja de departamentos dos marketplaces brasileiros. A Olist conecta pequenos negócios de todo o Brasil a canais de vendas, sem complicações e com um único contrato. Esses comerciantes conseguem vender seus produtos através da Olist Store e enviá-los diretamente aos clientes utilizando os parceiros logísticos da Olist. 

Após o cliente comprar um produto na Olist Store, o vendedor é notificado para atender a esse pedido. Uma vez que o cliente recebe o produto, ou que a data estimada de entrega expire, o cliente recebe uma pesquisa de satisfação por e-mail, onde pode dar uma nota para a experiência de compra e escrever alguns comentários.



<img src='https://i.imgur.com/HRhd2Y0.png' height=400px/>

#### Informações adcionais
----------

##### Conjunto de Dados de Clientes(olist_customer)
> Este conjunto de dados contém informações sobre o cliente e sua localização. Utilize-o para identificar clientes únicos no conjunto de dados de pedidos e para encontrar a localização de entrega dos pedidos.
>
> Em nosso sistema, cada pedido é atribuído a um customer_id exclusivo. Isso significa que o mesmo cliente terá diferentes ids para pedidos distintos. O propósito de ter um customer_unique_id no conjunto de dados é permitir que você identifique clientes que fizeram novas compras na loja. Caso contrário, você descobriria que cada pedido estaria associado a um cliente diferente.
--------

##### Conjunto de Dados de Geolocalização (geolocation)
> Este conjunto de dados contém informações sobre os códigos postais brasileiros e suas coordenadas de latitude/longitude. Utilize-o para plotar mapas e calcular distâncias entre vendedores e clientes

--------

##### Conjunto de Dados de Itens de Pedidos (order_items)
> Este conjunto de dados inclui informações sobre os itens comprados em cada pedido.

> Exemplo:
> O order_id = 00143d0f86d6fbd9f9b38ab440ac16f5 possui 3 itens (mesmo produto). Cada item tem o frete calculado de acordo com suas medidas e peso. Para obter o valor total do frete de cada pedido, basta somar.


-------
##### Payments Dataset
> This dataset includes data about the orders payment options.

-------
##### Conjunto de Dados de Avaliações de Pedidos (reviews dataset) 
> Este conjunto de dados inclui informações sobre as avaliações feitas pelos clientes.
> Após o cliente comprar um produto na Olist Store, o vendedor é notificado para atender a esse pedido. Quando o cliente recebe o produto, ou quando a data estimada de entrega se expira, o cliente recebe uma pesquisa de satisfação por e-mail, onde pode dar uma nota para a experiência de compra e escrever alguns comentários.

### Limpeza de dados

In [1]:
import polars as pl
import pandas as pd

In [6]:
cliente = pl.read_csv(
    source='dataset/olist_customers_dataset.csv', infer_schema=False
)

In [7]:
cliente.head(2)

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
str,str,str,str,str
"""06b8999e2fba1a1fbc88172c00ba8b…","""861eff4711a542e4b93843c6dd7feb…","""14409""","""franca""","""SP"""
"""18955e83d337fd6b2def6b18a428ac…","""290c77bc529b7ac935b93aa66c333d…","""09790""","""sao bernardo do campo""","""SP"""


In [14]:
cliente.shape

(99441, 5)

Verificando a quantidade de valores únicos de cada coluna

In [31]:
pd.DataFrame({
    col : cliente.select(pl.col(col)).n_unique() \
    for col in cliente.columns
}, index=['Quantidade de valores únicos']).T

,Quantidade de valores únicos
customer_id,99441
customer_unique_id,96096
customer_zip_code_prefix,14994
customer_city,4119
customer_state,27


> `customer_id` é uma chave primaria
>
> `customer_zip_code_prefix` é uma chave estrangeira

Chave primaria e chave estrangeira não necessitam de tratamento, somente, se houver valores nulos nesse conj. dados. Mais abaixo irei verificar a quantidade de valores nulos por colunas

In [49]:
cliente.select(pl.col('customer_id').null_count()).item()

0

In [63]:
cliente\
    .null_count()\
    .transpose(
        include_header=True,
        header_name   ='colunas',
        column_names  =['Quantidade de valores nulos']
).to_pandas()

,colunas,Quantidade de valores nulos
0,customer_id,0
1,customer_unique_id,0
2,customer_zip_code_prefix,0
3,customer_city,0
4,customer_state,0


> Não há valores nulos nas colunas acima